In [1]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
# from tensorflow.keras.optimizers import Adam


# # Load the dataset
# load_dir = './saved_dataset'
# loaded_dataset = tf.data.experimental.load(load_dir)

# batch_size = 32
# loaded_dataset = loaded_dataset.batch(batch_size).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

# number_of_labels = 10


Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


2024-03-08 15:07:14.805259: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-03-08 15:07:14.805280: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-08 15:07:14.805285: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-08 15:07:14.805317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-08 15:07:14.805331: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# len(loaded_dataset)

225

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Conv1D, BatchNormalization, LeakyReLU, concatenate, GlobalAveragePooling1D, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, optimizers


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)



# CNN Block
def cnn_block(x, filters, kernel_size, strides, activation):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    if activation == 'leaky_relu':
        x = LeakyReLU(alpha=0.1)(x)
    elif activation == 'relu':
        x = ReLU()(x)
    return x

# Parameters
num_frames = 45  # Adjust to match the fixed number of frames you've processed per video
num_features = 1024  # Feature dimension per frame
batch_size = 64
number_of_labels = 10

# Load the dataset
load_dir = './saved_dataset'
loaded_dataset = tf.data.experimental.load(load_dir)

# Shuffle and prepare the datasets
shuffled_dataset = loaded_dataset.shuffle(buffer_size=10000)
split_size = int(0.8 * len(shuffled_dataset))
train_dataset = shuffled_dataset.take(split_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
validation_dataset = shuffled_dataset.skip(split_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Input for the frame sequence
frame_input = Input(shape=(num_frames, num_features), name='frame_input')

# CNN Branch
cnn_branch = cnn_block(frame_input, filters=10, kernel_size=5, strides=1, activation='leaky_relu')
cnn_branch = cnn_block(cnn_branch, filters=10, kernel_size=5, strides=1, activation='leaky_relu')
cnn_branch = cnn_block(cnn_branch, filters=10, kernel_size=5, strides=1, activation='leaky_relu')
cnn_branch_output = GlobalAveragePooling1D()(cnn_branch)  # Aggregate the features from the CNN branch

# LSTM Branch
lstm_branch = LSTM(512, return_sequences=True)(frame_input)
lstm_branch = LSTM(256, return_sequences=True)(lstm_branch)
lstm_branch_output = LSTM(128, return_sequences=False)(lstm_branch)

# Transformer Branch
# transformer_branch = layers.Dense(512, activation="relu")(frame_input)  
# transformer_branch = TransformerBlock(512, 8, 64)(transformer_branch)
# transformer_branch = layers.GlobalAveragePooling1D()(transformer_branch)
# transformer_branch = layers.Dropout(0.5)(transformer_branch)
# transformer_branch_output = layers.Dense(128, activation="relu")(transformer_branch)

# Combine CNN and LSTM branch outputs and transformer branch output
combined_output = concatenate([cnn_branch_output, lstm_branch_output])

# Final Dense layers
x = Dense(128, activation='relu')(combined_output)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(number_of_labels, activation='sigmoid')(x)

# Create and compile the model
model = Model(inputs=frame_input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model with validation
model.fit(train_dataset, epochs=50, validation_data=validation_dataset)


Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


2024-03-08 15:57:33.374354: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-03-08 15:57:33.374367: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-08 15:57:33.374372: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-08 15:57:33.374402: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-08 15:57:33.374415: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 10), (None, 45, 256)]

In [2]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, optimizers

# class TransformerBlock(layers.Layer):
#     def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock, self).__init__()
#         self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
#         self.ffn = tf.keras.Sequential(
#             [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
#         )
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(rate)
#         self.dropout2 = layers.Dropout(rate)

#     def call(self, inputs, training):
#         attn_output = self.att(inputs, inputs)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         return self.layernorm2(out1 + ffn_output)

# def build_model(num_frames, num_features, num_classes):
#     inputs = layers.Input(shape=(num_frames, num_features))
#     x = layers.Dense(512, activation="relu")(inputs)
#     x = TransformerBlock(512, 8, 2048)(x)
#     x = layers.GlobalAveragePooling1D()(x)
#     x = layers.Dropout(0.5)(x)
#     outputs = layers.Dense(num_classes, activation="sigmoid")(x)

#     model = models.Model(inputs=inputs, outputs=outputs)
#     return model

# # Load and prepare your dataset
# load_dir = './saved_dataset'
# loaded_dataset = tf.data.experimental.load(load_dir)

# # Shuffle and split the dataset
# batch_size = 32
# shuffled_dataset = loaded_dataset.shuffle(10000)
# train_size = int(0.8 * len(shuffled_dataset))
# train_dataset = shuffled_dataset.take(train_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
# val_dataset = shuffled_dataset.skip(train_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# # Assuming each video sequence has been standardized to have a fixed number of frames (num_frames)
# # and each frame is represented by a 1024-dimensional vector
# num_frames = 45  # This is an example; adjust based on your actual data
# num_features = 1024
# num_classes = 9

# model = build_model(num_frames, num_features, num_classes)
# model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# model.summary()

# # Train the model
# model.fit(train_dataset, validation_data=val_dataset, epochs=10)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 45, 1024)]        0         
                                                                 
 dense_2 (Dense)             (None, 45, 512)           524800    
                                                                 
 transformer_block (Transfo  (None, 45, 512)           10503168  
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 512)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 9)                 4617  

2024-03-08 14:34:01.796173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-03-08 14:34:01.905096: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


180/180 [==============================] - 58s 307ms/step - loss: 0.3756 - accuracy: 0.2622 - val_loss: 0.3495 - val_accuracy: 0.2981
Epoch 2/10
180/180 [==============================] - 56s 302ms/step - loss: 0.3443 - accuracy: 0.2792 - val_loss: 0.3384 - val_accuracy: 0.3065
Epoch 3/10
180/180 [==============================] - 55s 300ms/step - loss: 0.3411 - accuracy: 0.2879 - val_loss: 0.3399 - val_accuracy: 0.3092
Epoch 4/10
  5/180 [..............................] - ETA: 1:00 - loss: 0.3460 - accuracy: 0.2375

KeyboardInterrupt: 